<a href="https://colab.research.google.com/github/FaridRasulov/NeuralODE/blob/master/Thesis_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchdiffeq
!pip install pytorch_utils
!pip install tqdm
!pip install adversarial-robustness-toolbox
!pip install kornia

In [13]:
import os
import argparse
import logging
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.functional as F
import torch.optim as optim
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

In [53]:
if adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm(min(32, dim), dim)


class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)

class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.norm1 = norm(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.norm2 = norm(planes)
        self.conv2 = conv3x3(planes, planes)

    def forward(self, x):
        shortcut = x

        out = self.relu(self.norm1(x))

        if self.downsample is not None:
            shortcut = self.downsample(out)

        out = self.conv1(out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + shortcut

class ConcatConv2d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True, transpose=False):
        super(ConcatConv2d, self).__init__()
        module = nn.ConvTranspose2d if transpose else nn.Conv2d
        self._layer = module(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)

class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out

class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=tol, atol=tol)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value


device = torch.device('cuda:' + str(gpu) if torch.cuda.is_available() else 'cpu')
is_odenet = network == 'odenet'

if downsampling_method == 'conv':
        downsampling_layers = [
            nn.Conv2d(1, 64, 3, 1),
            norm(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 4, 2, 1),
            norm(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 4, 2, 1),
        ]
elif downsampling_method == 'res':
        downsampling_layers = [
            nn.Conv2d(1, 64, 3, 1),
            ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
            ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
        ]

feature_layers = [ODEBlock(ODEfunc(64))] if is_odenet else [ResBlock(64, 64) for _ in range(6)]
fc_layers = [norm(64), nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1)), Flatten(), nn.Linear(64, 10)]

In [15]:
parser = argparse.ArgumentParser()
network = 'odenet' # 'resnet'
tol = 1e-3
adjoint = False # True
downsampling_method = 'conv' # 'res'
nepochs = 2
data_aug = True # False
lr = 0.1
batch_size = 128
test_batch_size = 1000
save = './experiment1'
debug = 'store_true'
gpu = 0

In [16]:
# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()
print('Step 1: Load the MNIST dataset')
# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)
print("Step 1a: Swap axes to PyTorch's NCHW format")

Step 1: Load the MNIST dataset
Step 1a: Swap axes to PyTorch's NCHW format


In [58]:
# Step 2a: Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
print('Step 2a: Define the loss function and the optimizer')
# Step 3: Create the ART classifier

model_sequenced = nn.Sequential(*downsampling_layers, *feature_layers, *fc_layers).to(device)
model_shuffled = nn.Sequential(*downsampling_layers, *feature_layers, *fc_layers).to(device)

classifier_for_shuffled = PyTorchClassifier(
    model=model_sequenced,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)
classifier_for_sequenced = PyTorchClassifier(
    model=model_shuffled,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)
print('Step 3: Create the ART classifier')

Step 2a: Define the loss function and the optimizer
Step 3: Create the ART classifier


In [18]:
# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)
x_train_adv = attack.generate(x=x_train)
print('Step 6: Generate adversarial test examples')

Step 6: Generate adversarial test examples


In [59]:
from sklearn.utils import shuffle
sequenced_train_set, sequenced_label_set = np.concatenate((x_train, x_train_adv), axis=0), np.concatenate((y_train, y_train), axis=0)
shuffled_train_set, shuffled_label_set = shuffle(np.concatenate((x_train, x_train_adv), axis=0), np.concatenate((y_train, y_train), axis=0), random_state=0)
print(sequenced_train_set.shape, sequenced_label_set.shape)
print(shuffled_train_set.shape, shuffled_label_set.shape)

(120000, 1, 28, 28) (120000, 10)
(120000, 1, 28, 28) (120000, 10)


In [ ]:
classifier_for_sequenced.fit(sequenced_train_set, sequenced_label_set, batch_size=64, nb_epochs=nepochs)
print('Trained on an sequenced adv trainset')
classifier_for_shuffled.fit(shuffled_train_set, shuffled_label_set, batch_size=64, nb_epochs=nepochs)
print('Trained on an shuffled adv trainset')

In [57]:
# Step 7: Evaluate the ART classifier on adversarial test examples
predictions_sequenced = classifier_for_sequenced.predict(x_test)
predictions_shuffled = classifier_for_shuffled.predict(x_test)
accuracy_sequenced = np.sum(np.argmax(predictions_sequenced, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
accuracy_shuffled = np.sum(np.argmax(predictions_shuffled, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adv sequenced test examples: {}%".format(accuracy_sequenced * 100))
print("Accuracy on adv shuffled test examples: {}%".format(accuracy_shuffled * 100))
print('Step 7: Evaluate the ART classifier on adversarial test examples')

Accuracy on adv sequenced test examples: 11.35%
Accuracy on adv shuffled test examples: 11.35%
Step 7: Evaluate the ART classifier on adversarial test examples
